In [ ]:
%load_ext aiida
%aiida

In [1]:
from aiida.orm import FolderData
from aiida.engine import calcfunction
import ipywidgets as widgets
from IPython.display import display, clear_output
import tempfile
import os

person_dropdown = widgets.Dropdown(
    options=['cpi', 'fas', 'psd'],
    value='cpi',
    description='Person:',
    style={'description_width': 'initial'}
)

# DOI Metadata Function (Assumed)
def get_doi_metadata(doi):
    import requests
    url = f"https://api.crossref.org/works/{doi}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        data = response.json()
        title = data['message'].get('title', ['No title found'])[0]
        authors = data['message'].get('author', [])
        author_names = [f"{author.get('given', '')} {author.get('family', '')}".strip() for author in authors]
        journal = data['message'].get('container-title', ['No journal found'])[0]
        return {'title': title, 'authors': author_names, 'journal': journal}
    except Exception as e:
        return {'title': f"Error: {e}", 'authors': [], 'journal': None}

# Widgets for form
doi_input = widgets.Text(
    value='',
    placeholder='Enter DOI',
    description='DOI:',
    style={'description_width': 'initial'}
)

metadata_output = widgets.Output()

# Check DOI Button
def check_metadata(_):
    with metadata_output:
        clear_output()
        doi = doi_input.value.strip()
        if not doi:
            print("Please enter a valid DOI.")
        else:
            metadata = get_doi_metadata(doi)
            print("Title:", metadata['title'])
            print("Authors:", ', '.join(metadata['authors']))
            print("Journal:", metadata['journal'])

check_button = widgets.Button(
    description='Check DOI',
    button_style='info'
)
check_button.on_click(check_metadata)

# Add Property Button
additional_properties = widgets.VBox()

def add_property_field(_):
    label_input = widgets.Text(
        placeholder='Enter property name (e.g., "bandgap")',
        description='Property:',
        style={'description_width': 'initial'}
    )
    value_input = widgets.Text(
        placeholder='Enter property value (e.g., "1.1 eV")',
        description='Value:',
        style={'description_width': 'initial'}
    )
    additional_properties.children += (widgets.HBox([label_input, value_input]),)

add_property_button = widgets.Button(
    description='Add Property',
    button_style='success'
)
add_property_button.on_click(add_property_field)

# Add File Upload Button
uploaded_files = widgets.VBox()

def add_file_upload_field(_):
    file_upload = widgets.FileUpload(
        accept='*',
        multiple=False,
        description='Upload File'
    )
    description_dropdown = widgets.Dropdown(
        options=['Image preview', 'spectra', 'other'],
        value='other',
        description='Type:',
        style={'description_width': 'initial'}
    )
    description_text = widgets.Text(
        placeholder='Further description (optional)',
        description='Details:',
        style={'description_width': 'initial'}
    )
    uploaded_files.children += (widgets.VBox([file_upload, description_dropdown, description_text]),)

add_file_upload_button = widgets.Button(
    description='Add File Upload',
    button_style='primary'
)
add_file_upload_button.on_click(add_file_upload_field)

# FolderData Creation @calcfunction
@calcfunction
def create_folderdata(properties, comments, uploaded_files):
    folder_data = FolderData()
    temp_dir = tempfile.mkdtemp()

    # Store properties
    properties_file = os.path.join(temp_dir, "properties.txt")
    with open(properties_file, 'w') as prop_file:
        for label, value in properties.get_dict().items():
            prop_file.write(f"{label}: {value}\n")
    folder_data.put_object_from_file(properties_file, "properties.txt")

    # Store comments
    comments_file = os.path.join(temp_dir, "comments.txt")
    with open(comments_file, 'w') as comm_file:
        comm_file.write(comments.value)
    folder_data.put_object_from_file(comments_file, "comments.txt")

    # Store uploaded files
    for file_info in uploaded_files.get_list():
        file_content, file_type, details = file_info
        temp_file_path = os.path.join(temp_dir, file_content['filename'])
        with open(temp_file_path, 'wb') as temp_file:
            temp_file.write(file_content['content'])
        folder_data.put_object_from_file(temp_file_path, file_content['filename'])

    return folder_data

# Store Record Button
comments_field = widgets.Textarea(
    value='',
    placeholder='Enter any comments',
    description='Comments:',
    layout=widgets.Layout(width='100%', height='100px'),
    style={'description_width': 'initial'}
)

def store_record(_):
    with metadata_output:
        clear_output()
        # Extract properties
        properties_dict = {}
        for prop in additional_properties.children:
            label = prop.children[0].value.strip()
            value = prop.children[1].value.strip()
            if label and value:
                properties_dict[label] = value

        # Extract uploaded files
        uploaded_files_list = []
        for file_widget in uploaded_files.children:
            file_upload = file_widget.children[0]
            description_type = file_widget.children[1].value
            description_text = file_widget.children[2].value.strip()
            for filename, content in file_upload.value.items():
                uploaded_files_list.append((content, description_type, description_text))

        # Create FolderData
        properties = Dict(dict=properties_dict)
        comments = Str(comments_field.value)
        uploaded_files_data = List(list=uploaded_files_list)
        folder_data = create_folderdata(properties, comments, uploaded_files_data)

        print(f"FolderData created with PK: {folder_data.pk}")

store_button = widgets.Button(
    description='Store Record',
    button_style='success'
)
store_button.on_click(store_record)

# Organize Layout
form = widgets.VBox([
    doi_input,
    person_dropdown,
    check_button,
    metadata_output,
    widgets.HBox([add_property_button]),
    additional_properties,
    comments_field,
    widgets.HBox([add_file_upload_button]),
    uploaded_files,
    widgets.HBox([store_button])
])

display(form)
